# Advanced Robotics 
This file is written by:
1. Ahmad Hamdan
2. Ghadeer Issa

In [2]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

#%matplotlib notebook
%matplotlib Qt
#%matplotlib widgets

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


![title](img/banda.png)

## DH Parameters

In [3]:
robot = rtb.models.DH.Panda()                  # instantiate robot model
print(robot)

DHRobot: Panda (by Franka Emika), 7 joints (RRRRRRR), dynamics, geometry, modified DH parameters
┌────────┬────────┬─────┬───────┬─────────┬────────┐
│ aⱼ₋₁   │  ⍺ⱼ₋₁  │ θⱼ  │  dⱼ   │   q⁻    │   q⁺   │
├────────┼────────┼─────┼───────┼─────────┼────────┤
│    0.0 │   0.0° │  q1 │ 0.333 │ -166.0° │ 166.0° │
│    0.0 │ -90.0° │  q2 │   0.0 │ -101.0° │ 101.0° │
│    0.0 │  90.0° │  q3 │ 0.316 │ -166.0° │ 166.0° │
│ 0.0825 │  90.0° │  q4 │   0.0 │ -176.0° │  -4.0° │
│-0.0825 │ -90.0° │  q5 │ 0.384 │ -166.0° │ 166.0° │
│    0.0 │  90.0° │  q6 │   0.0 │   -1.0° │ 215.0° │
│  0.088 │  90.0° │  q7 │ 0.107 │ -166.0° │ 166.0° │
└────────┴────────┴─────┴───────┴─────────┴────────┘

┌─────┬──────┐
│tool │ None │
└─────┴──────┘

┌─────┬─────┬────────┬─────┬───────┬─────┬───────┬──────┐
│name │ q0  │ q1     │ q2  │ q3    │ q4  │ q5    │ q6   │
├─────┼─────┼────────┼─────┼───────┼─────┼───────┼──────┤
│  qr │  0° │ -17.2° │  0° │ -126° │  0° │  115° │  45° │
│  qz │  0° │  0°    │  0° │  0°   │  0° 

The first table shows the kinematic parameters, and from the column titles we can see clearly that this is expressed in terms of standard Denavit-Hartenberg parameters.  The first column shows that the joint variables qi are rotations since they are in the θ column.  Joint limits are also shown.  Joint flip (motion in the opposite sense) would be indicated by the joint variable being shown as for example like `-q3`.

The second table shows some named joint configurations.  For example `ur10.qr`

## Robot Model

In [4]:
robot = rtb.models.Panda()                  # instantiate robot model
print(robot)

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

In [27]:
robot.plot(robot.qr)

Swift backend, t = 0.05, scene:
  panda

## Forward Kinematics

In [5]:
T = robot.fkine([-0.5, -0.5, -0.4, -0.4, -0.5, 0.6, 0.3])  # forward kinematics
print(T)

   0.5099   -0.8165    0.2709   -0.1506    
  -0.6557   -0.5727   -0.492    -0.1306    
   0.5568    0.07326  -0.8274    0.8901    
   0         0         0         1         



## Inverse
```ikine_LM ```is a generalised iterative numerical solution based on Levenberg-Marquadt minimization, and additional status results are also returned as part of a named tuple.

In [6]:
sol = robot.ikine_LM(T)                     # inverse kinematics
print(sol)

IKsolution(q=array([ -0.5582,  -0.4728,   -0.495,  -0.3804,  -0.3841,   0.6033,   0.2596]), success=True, reason=None, iterations=18, residual=7.920168289901014e-11)


## Plot 
Plotting the robot model and configuration using swift


In [24]:
robot.plot(sol.q, backend='swift')

Swift backend, t = 0.05, scene:
  panda

## Jacobian

Calculating Jacbian and simulating the movement between 2 points.

In [7]:
Je = robot.jacobe([0,0,0,0,0,0, 0])  
print(Je)

[[ 0.06223   0.3462  0.06223  -0.1228  0.06223   0.1488        0]
 [-0.06223   0.3462 -0.06223  -0.1228 -0.06223   0.1488        0]
 [       0    0.088        0  -0.0055        0   -0.088        0]
 [       0   0.7071        0  -0.7071        0  -0.7071        0]
 [       0  -0.7071        0   0.7071        0   0.7071        0]
 [      -1        0       -1        0       -1        0        1]]


In [8]:
#simulating trajectory between 2 configurations
traj = rtb.jtraj(robot.qr, sol.q, 100)
robot.plot(traj.q, backend='swift')

connection handler failed
Traceback (most recent call last):
  File "/home/ahmad/.local/lib/python3.10/site-packages/websockets/legacy/server.py", line 240, in handler
    await self.ws_handler(self)
  File "/home/ahmad/.local/lib/python3.10/site-packages/websockets/legacy/server.py", line 1186, in _ws_handler
    return await cast(
  File "/home/ahmad/.local/lib/python3.10/site-packages/swift/SwiftRoute.py", line 134, in serve
    await self.expect_message(websocket, expected)
  File "/home/ahmad/.local/lib/python3.10/site-packages/swift/SwiftRoute.py", line 139, in expect_message
    recieved = await websocket.recv()
  File "/home/ahmad/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 568, in recv
    await self.ensure_open()
  File "/home/ahmad/.local/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 944, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: received 1001 (going away); then sent 1001 (

KeyboardInterrupt: 

## Dynamics

In [8]:
robot = rtb.models.DH.Panda()
#print(robot)
tau = robot.rne(robot.qr, np.zeros((7,)), np.zeros((7,)))
print(tau)

[       0    -10.6        0    14.59        0   0.7089        0]


In [28]:
robot.gravload(robot.qr)

array([       0,    -10.6,        0,    14.59,        0,   0.7089,        0])

In [9]:
robot.inertia(robot.qr)

ValueError: operands could not be broadcast together with shapes (6,) (7,) 

The velocity terms are a bit harder to comprehend but they mean that rotation of one joint (and its link) can exert a torque on other joints. Consider that the should joint is rotating at 1 rad/sec, then the torque will be

# Calibration


In [11]:
from spatialmath import SE3
E1 = rtb.ET.tz(0.333)
E2 = rtb.ET.Rz()
E3 = rtb.ET.Ry()
E4 = rtb.ET.tz(0.316)
E5 = rtb.ET.Rz()
E6 = rtb.ET.tx(0.0825)
E7 = rtb.ET.Ry(flip=True)
E8 = rtb.ET.tx(-0.0825)
E9 = rtb.ET.tz(0.384)
E10 = rtb.ET.Rz()
E11 = rtb.ET.Ry(flip=True)
E12 = rtb.ET.tx(0.088)
E13 = rtb.ET.Rx(np.pi)
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()
robot = rtb.models.DH.Panda()

# We can make an ETS representing a Panda by incorprating all 15 ETs into an ETS
panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15

# View the ETS
print(panda)
print()

q1=0
q2=0
q3=0
q4=0
q5=0
q6=0
q7=0

d0 = 0.333
d1 = 0.316
d2 = 0.0825
d3 = -0.0825
d4= 0.384
d5 = 0.088
d6 = 210

E1 = SE3.Tz(0.333)
E2 = SE3.Rz(q1)
E3 = SE3.Ry(q2)
E4 = SE3.Tz(0.316)
E5 = SE3.Rz(q3)
E6 = SE3.Tx(0.0825)
E7 = SE3.Ry(-q4)
E8 = SE3.Tx(-0.0825)
E9 = SE3.Tz(0.384)
E10 = SE3.Rz(q5)
E11 = SE3.Ry(-q6)
E12 = SE3.Tx(0.088)
E13 = SE3.Rx(np.pi)
E14 = SE3.Tz(0.210)
E15 = SE3.Rz((-np.pi/4) + q7) 

panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15
print(panda)

print(robot.fkine([0, 0, 0, -0, 0, -0, 0]))



tz(0.333) ⊕ Rz(q0) ⊕ Ry(q1) ⊕ tz(0.316) ⊕ Rz(q2) ⊕ tx(0.0825) ⊕ Ry(-q3) ⊕ tx(-0.0825) ⊕ tz(0.384) ⊕ Rz(q4) ⊕ Ry(-q5) ⊕ tx(0.088) ⊕ Rx(180°) ⊕ tz(0.107) ⊕ Rz(q6)

   0.7071    0.7071    0         0.088     
   0.7071   -0.7071    0         0         
   0         0        -1         0.823     
   0         0         0         1         

   0.7071    0.7071    0         0.088     
   0.7071   -0.7071    0         0         
   0         0        -1         0.823     
   0         0         0         1         



### Utils
a collection of tranformation matrix

In [12]:
cos=np.cos
sin=np.sin
pi=np.pi


def Rxd(phi):
    rx = np.array([[ 0,         0,         0, 0],
                   [0, -sin(phi),  -cos(phi), 0],
                   [0, cos(phi), -sin(phi), 0],
                   [0,         0,         0, 0]])
    return SE3(rx, check=False)

def Ryd(phi):
    ry = np.array([[ -sin(phi), 0,  cos(phi), 0],
                   [0,         0,         0, 0],
                   [-cos(phi), 0, -sin(phi), 0],
                   [0,         0,         0, 0]])
    return SE3(ry, check=False)

def Rzd(phi):
    rz = np.array([[ -sin(phi),  -cos(phi), 0, 0],
                   [ cos(phi), -sin(phi), 0, 0],
                   [0,         0,         0, 0],
                   [0,         0,         0, 0]])
    return SE3(rz, check=False)
Txd0 = np.array([[0,   0,  0,  1],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0]])
Tyd0=np.array([[0,   0,  0,  0],
                   [0,   0,  0,  1],
                   [0,   0,  0,  0],
                   [0,   0,  0,  0]])
Tzd0= np.array([[0,   0,  0,  0],
                   [0,   0,  0,  0],
                   [0,   0,  0,  1],
                   [0,   0,  0,  0]])

Txd=SE3(Txd0, check=False)
Tyd=SE3(Tyd0, check=False)
Tzd=SE3(Tzd0, check=False)



Tx=SE3.Tx
Ty=SE3.Ty
Tz=SE3.Tz
Rx=SE3.Rx
Ry=SE3.Ry
Rz=SE3.Rz


## Irreducable model derivation:

$T=[T_xT_yT_zR_xR_yR_z]_b.
R_z(q_1+\Delta q_1).[T_xT_yR_xR_y].
R_y(q_2+\Delta q_2).[T_xT_zR_xR_z].
R_z(q_3+\Delta q_3).[T_xT_yR_xR_y].
R_y(q_4+\Delta q_4).[T_xT_zR_xR_z].
R_z(q_5+\Delta q_5).[T_xT_yR_xR_y].
R_y(q_6+\Delta q_6).[T_xT_zR_xR_z].
R_z(q_7+\Delta q_7).[T_xT_yR_xR_y].
[T_xT_yT_zR_xR_yR_z]_t$




$T=[T_xT_yT_zR_xR_yR_z]_b.
R_z(q_1+\cancel{\Delta q_1}).[T_xT_yR_x\cancel{R_y}].
R_y(q_2+\Delta q_2).[T_xT_zR_x\cancel{R_z}].
R_z(q_3+\Delta q_3).[T_xT_yR_x\cancel{R_y}].
R_y(q_4+\Delta q_4).[T_xT_zR_x\cancel{R_z}].
R_z(q_5+\Delta q_5).[T_xT_yR_x\cancel{R_y}].
R_y(q_6+\Delta q_6).[T_x\cancel{T_z}R_x\cancel{R_z}].
R_z(q_7+\cancel{\Delta q_7}).[\cancel{T_x}\cancel{T_y}\cancel{R_x}\cancel{R_y}].
[\cancel{T_x}\cancel{T_y}\cancel{T_z}\cancel{R_x}\cancel{R_y}\cancel{R_z}]_t$




$T_{robot}=R_z(q_1).[T_xT_yR_x].
R_y(q_2+\Delta q_2).[T_xT_zR_x].
R_z(q_3+\Delta q_3).[T_xT_yR_x].
R_y(q_4+\Delta q_4).[T_xT_zR_x].
R_z(q_5+\Delta q_5).[T_xT_yR_x].
R_y(q_6+\Delta q_6).[T_xR_x].
R_z(q_7)$


$\mathbf{\pi}=\{ p_{x1} \quad p_{y1} \quad \varphi_{x1} 
\quad \Delta q_2 \quad p_{x2} \quad p_{z2} \quad \varphi_{x2}
\quad \Delta q_3 \quad p_{x3} \quad p_{y3} \quad \varphi_{y3} 
\quad \Delta q_4 \quad p_{x4} \quad p_{z4} \quad \varphi_{x4} 
\quad \Delta q_5 \quad p_{x5} \quad p_{y5} \quad \varphi_{x5} 
\quad \Delta q_6 \quad p_{x6}  \quad \varphi_{x6} \}$


$\mathbf{\pi}_0=\{ 0 \quad 0 \quad 0 \quad 0 \quad 0 \quad d_1 \quad 0 \quad 0 
\quad d_2\quad 0 \quad 0 \quad 0 \quad d_3 \quad d_4 \quad 0 \quad 0 
\quad 0 \quad 0 \quad 0 \quad 0 \quad d_5 \quad 0 
\}$

in case of $\mathbf{\pi}_0$ the equation should be as follows:

$T_{robot}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$



$\Delta q_2: T_{1}^{'}=T_{base}.[R_z(q_1).H_{Ry}^{'}.R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  


$p_{z1}: T_{2}^{'}=T_{base}.[R_z(q_1).R_y(q_2).H_{Tz}^{'}.R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  



$\Delta q_3: T_{3}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).H_{Rz}^{'}.R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  



$p_{x2}: T_{4}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).H_{Tx}^{'}.R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  





$\Delta q_4: T_{5}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).H_{Ry}^{'}.R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  



$p_{x3}: T_{6}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).H_{Tx}^{'}.T_z(d4).R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  
 



$p_{z3}: T_{7}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).H_{Tz}^{'}.R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  





$\Delta q_5: T_{8}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).H_{Rz}^{'}.R_z(q_5).R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  
 

$\Delta q_6: T_{9}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).H_{Ry}^{'}.R_y(q_6).T_x(d5).R_z(q_7)]$

<br>  

$p_{x5}: T_{10}^{'}=T_{base}.[R_z(q_1).R_y(q_2).T_z(d1).R_z(q_3).T_x(d2).R_y(q_4).T_x(d3).T_z(d4).R_z(q_5).R_y(q_6).H_{Tx}^{'}.R_z(q_7)]$


In [28]:

def parameteric_jacobian(Q):
    q1, q2, q3, q4, q5, q6, q7 =Q



    T_base = Tz(d0)
    T_robot = T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T_robot_inv=np.linalg.inv(T_robot)

    T1_=T_base * Rz(q1) * Ryd(0) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7) # q2
    T1_np=np.array(T1_)
    T1_r=T1_np@T_robot_inv

    Jth1 = np.array([[T1_np[0,3], T1_np[1,3], T1_np[2,3], T1_r[2,1], T1_r[0,2], T1_r[1,0]]]).T
    
    T2_=T_base * Rz(q1) * Ry(q2) * Tzd * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7) # d1
    T2_np=np.array(T2_)
    T2_r=T2_np@T_robot_inv
    Jth2 = np.array([[T2_np[0,3], T2_np[1,3], T2_np[2,3], T2_r[2,1], T2_r[0,2], T2_r[1,0]]]).T

    T3_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rzd(0) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T3_np=np.array(T3_)
    T3_r=T3_np@T_robot_inv

    Jth3 = np.array([[T3_np[0,3], T3_np[1,3], T3_np[2,3], T3_r[2,1], T3_r[0,2], T3_r[1,0]]]).T

    T4_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Txd * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T4_np=np.array(T4_)
    T4_r=T4_np@T_robot_inv

    Jth4 = np.array([[T4_np[0,3], T4_np[1,3], T4_np[2,3], T4_r[2,1], T4_r[0,2], T4_r[1,0]]]).T

    T5_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ryd(0) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T5_np=np.array(T5_)
    T5_r=T5_np@T_robot_inv

    Jth5 = np.array([[T5_np[0,3], T5_np[1,3], T5_np[2,3], T5_r[2,1], T5_r[0,2], T5_r[1,0]]]).T

    T6_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Txd * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T6_np=np.array(T6_)
    T6_r=T6_np@T_robot_inv

    Jth6 = np.array([[T6_np[0,3], T6_np[1,3], T6_np[2,3], T6_r[2,1], T6_r[0,2], T6_r[1,0]]]).T

    T7_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tzd * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T7_np=np.array(T7_)
    T7_r=T7_np@T_robot_inv

    Jth7 = np.array([[T7_np[0,3], T7_np[1,3], T7_np[2,3], T7_r[2,1], T7_r[0,2], T7_r[1,0]]]).T

    T8_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rzd(0) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)
    T8_np=np.array(T8_)
    T8_r=T8_np@T_robot_inv

    Jth8 = np.array([[T8_np[0,3], T8_np[1,3], T8_np[2,3], T8_r[2,1], T8_r[0,2], T8_r[1,0]]]).T

    T9_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ryd(0) * Ry(q6) * Tx(d5) * Rz(q7)
    T9_np=np.array(T9_)
    T9_r=T9_np@T_robot_inv

    Jth9 = np.array([[T9_np[0,3], T9_np[1,3], T9_np[2,3], T9_r[2,1], T9_r[0,2], T9_r[1,0]]]).T

    T10_=T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Txd * Rz(q7)
    T10_np=np.array(T10_)
    T10_r=T10_np@T_robot_inv

    Jth10 = np.array([[T10_np[0,3], T10_np[1,3], T10_np[2,3], T10_r[2,1], T10_r[0,2], T10_r[1,0]]]).T

    
    J_para=np.hstack((Jth1,Jth2,Jth3,Jth4,Jth5,Jth6,Jth7,Jth8,Jth9,Jth10))

    return J_para


J = parameteric_jacobian([0, 0, 0, 0, 0, 0, 0])
print(J)

[[     0.7        0        0        1    0.384        1        0        0        0        1]
 [       0        0    0.088        0        0        0        0    0.088        0        0]
 [  -0.088        1        0        0  -0.0055        0        1        0   -0.088        0]
 [       0        0        0        0        0        0        0        0        0        0]
 [       1        0        0        0        1        0        0        0        1        0]
 [       0        0        1        0        0        0        0        1        0        0]]


In [16]:
def parameteric_jacobian_def(Q):
    q1, q2, q3, q4, q5, q6, q7 =Q
    
    d0 = 0.333 
    d1 = 0.316
    d2 = 0.0825
    d3 = -0.0825
    d4= 0.384
    d5 = 0.088
    d6 = 210

    d0_n = d0 - d0/100
    d1_n = d1 - d1/100
    d2_n = d2 - d2/100
    d3_n = d3 - d3/100
    d4_n = d4 - d4/100
    d5_n = d5 - d5/100
    d6_n = d6 - d6/100

    T_base = Tz(d0)
    T_robot = T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    
    T1_=T_base * Rz(q1) * Ryd(0) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7) # q2
    T1_np=np.array(T1_)
    Jth1 = np.array([[T1_np[0,3], T1_np[1,3], T1_np[2,3], T1_np[2,1], T1_np[0,2], T1_np[1,0]]]).T
    
    T2_=T_base * Rz(q1) * Ry(q2) * Tzd * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7) # d1
    T2_np=np.array(T2_)
    Jth2 = np.array([[T2_np[0,3], T2_np[1,3], T2_np[2,3], T2_np[2,1], T2_np[0,2], T2_np[1,0]]]).T

    T3_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rzd(0) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T3_np=np.array(T3_)
    Jth3 = np.array([[T3_np[0,3], T3_np[1,3], T3_np[2,3], T3_np[2,1], T3_np[0,2], T3_np[1,0]]]).T

    T4_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Txd * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T4_np=np.array(T4_)
    Jth4 = np.array([[T4_np[0,3], T4_np[1,3], T4_np[2,3], T4_np[2,1], T4_np[0,2], T4_np[1,0]]]).T

    T5_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ryd(0) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T5_np=np.array(T5_)
    Jth5 = np.array([[T5_np[0,3], T5_np[1,3], T5_np[2,3], T5_np[2,1], T5_np[0,2], T5_np[1,0]]]).T

    T6_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Txd * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T6_np=np.array(T6_)
    Jth6 = np.array([[T6_np[0,3], T6_np[1,3], T6_np[2,3], T6_np[2,1], T6_np[0,2], T6_np[1,0]]]).T

    T7_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tzd * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T7_np=np.array(T7_)
    Jth7 = np.array([[T7_np[0,3], T7_np[1,3], T7_np[2,3], T7_np[2,1], T7_np[0,2], T7_np[1,0]]]).T

    T8_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rzd(0) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T8_np=np.array(T8_)
    Jth8 = np.array([[T8_np[0,3], T8_np[1,3], T8_np[2,3], T8_np[2,1], T8_np[0,2], T8_np[1,0]]]).T

    T9_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ryd(0) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T9_np=np.array(T9_)
    Jth9 = np.array([[T9_np[0,3], T9_np[1,3], T9_np[2,3], T9_np[2,1], T9_np[0,2], T9_np[1,0]]]).T

    T10_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Txd * Rz(q7)
    T10_np=np.array(T10_)
    Jth10 = np.array([[T10_np[0,3], T10_np[1,3], T10_np[2,3], T10_np[2,1], T10_np[0,2], T10_np[1,0]]]).T

    
    J_para=np.hstack((Jth1,Jth2,Jth3,Jth4,Jth5,Jth6,Jth7,Jth8,Jth9,Jth10))
    #print(J@J.T)
    return J_para



J1 = parameteric_jacobian_def([0, 0, 0, 0, 0, 0, 0])
print(J1)

[[   0.693        0        0        1   0.3802        1        0        0        0        1]
 [       0        0  0.08712        0        0        0        0  0.08712        0        0]
 [-0.08712        1        0        0 -0.005445        0        1        0 -0.08712        0]
 [       0        0        0        0        0        0        0        0        0        0]
 [       1        0        0        0        1        0        0        0        1        0]
 [       0        0        1        0        0        0        0        1        0        0]]


In [33]:
d0_n = d0 #- d0/100
d1_n = d1 - d1/100
d2_n = d2 - d2/100
d3_n = d3 - d3/100
d4_n = d4 - d4/100
d5_n = d5 - d5/100
d6_n = d6 #- d6/100

def parameteric_jacobian(Q):
    q1, q2, q3, q4, q5, q6, q7 =Q



    T_base = Tz(d0)
    T_robot = T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T_robot_inv=np.linalg.inv(T_robot)

    T1_=T_base * Rz(q1) * Ryd(0) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7) # q2
    T1_np=np.array(T1_)
    T1_r=T1_np@T_robot_inv

    Jth1 = np.array([[T1_np[0,3], T1_np[1,3], T1_np[2,3], T1_r[2,1], T1_r[0,2], T1_r[1,0]]]).T
    
    T2_=T_base * Rz(q1) * Ry(q2) * Tzd * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7) # d1
    T2_np=np.array(T2_)
    T2_r=T2_np@T_robot_inv
    Jth2 = np.array([[T2_np[0,3], T2_np[1,3], T2_np[2,3], T2_r[2,1], T2_r[0,2], T2_r[1,0]]]).T

    T3_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rzd(0) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T3_np=np.array(T3_)
    T3_r=T3_np@T_robot_inv

    Jth3 = np.array([[T3_np[0,3], T3_np[1,3], T3_np[2,3], T3_r[2,1], T3_r[0,2], T3_r[1,0]]]).T

    T4_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Txd * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T4_np=np.array(T4_)
    T4_r=T4_np@T_robot_inv

    Jth4 = np.array([[T4_np[0,3], T4_np[1,3], T4_np[2,3], T4_r[2,1], T4_r[0,2], T4_r[1,0]]]).T

    T5_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ryd(0) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T5_np=np.array(T5_)
    T5_r=T5_np@T_robot_inv

    Jth5 = np.array([[T5_np[0,3], T5_np[1,3], T5_np[2,3], T5_r[2,1], T5_r[0,2], T5_r[1,0]]]).T

    T6_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Txd * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T6_np=np.array(T6_)
    T6_r=T6_np@T_robot_inv

    Jth6 = np.array([[T6_np[0,3], T6_np[1,3], T6_np[2,3], T6_r[2,1], T6_r[0,2], T6_r[1,0]]]).T

    T7_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tzd * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T7_np=np.array(T7_)
    T7_r=T7_np@T_robot_inv

    Jth7 = np.array([[T7_np[0,3], T7_np[1,3], T7_np[2,3], T7_r[2,1], T7_r[0,2], T7_r[1,0]]]).T

    T8_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rzd(0) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T8_np=np.array(T8_)
    T8_r=T8_np@T_robot_inv

    Jth8 = np.array([[T8_np[0,3], T8_np[1,3], T8_np[2,3], T8_r[2,1], T8_r[0,2], T8_r[1,0]]]).T

    T9_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ryd(0) * Ry(q6) * Tx(d5_n) * Rz(q7)
    T9_np=np.array(T9_)
    T9_r=T9_np@T_robot_inv

    Jth9 = np.array([[T9_np[0,3], T9_np[1,3], T9_np[2,3], T9_r[2,1], T9_r[0,2], T9_r[1,0]]]).T

    T10_=T_base * Rz(q1) * Ry(q2) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Txd * Rz(q7)
    T10_np=np.array(T10_)
    T10_r=T10_np@T_robot_inv

    Jth10 = np.array([[T10_np[0,3], T10_np[1,3], T10_np[2,3], T10_r[2,1], T10_r[0,2], T10_r[1,0]]]).T

    
    J_para=np.hstack((Jth1,Jth2,Jth3,Jth4,Jth5,Jth6,Jth7,Jth8,Jth9,Jth10))

    return J_para


J = parameteric_jacobian([0, 0, 0, 0, 0, 0, 0])
print(J)

[[  0.6968        0        0        1   0.3802        1        0        0        0        1]
 [       0        0  0.08712        0        0        0        0  0.08712        0        0]
 [-0.08712        1        0        0 -0.005445        0        1        0 -0.08712        0]
 [       0        0        0        0        0        0        0        0        0        0]
 [       1        0        0        0        1        0        0        0        1        0]
 [       0        0        1        0        0        0        0        1        0        0]]


In [49]:
d0 = 0.333 
d1 = 0.316
d2 = 0.0825
d3 = -0.0825
d4= 0.384
d5 = 0.088
d6 = 210

d0_n = d0 #- d0/100
d1_n = d1 - d1/100
d2_n = d2 - d2/100
d3_n = d3 - d3/100
d4_n = d4 - d4/100
d5_n = d5 - d5/100
d6_n = d6 - d6/100

T_base = Tz(d0)
T_robot = T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)    

ql = 0#-pi/16
qu = 2*pi
D_pi=[]

#
pi_d2=0
pi_d3=0
pi_d4=0
pi_d5=0

pi_q2=0
pi_q3=0
pi_q4=0
pi_q5=0

#
sum_pi_d2=0
sum_pi_d3=0
sum_pi_d4=0
sum_pi_d5=0

sum_pi_q2=0
sum_pi_q3=0
sum_pi_q4=0
sum_pi_q5=0


#

Delta_ps=[]
JJ=[]
criteira=0
for i in range(2):
    q1 =ql + (qu -ql)*(np.random.rand(1)[0])
    q2 =ql + (qu -ql)*(np.random.rand(1)[0])
    q3 =ql + (qu -ql)*(np.random.rand(1)[0])
    q4 = ql + (qu -ql)*(np.random.rand(1)[0])
    q5 = ql + (qu -ql)*(np.random.rand(1)[0])
    q6 = ql + (qu -ql)*(np.random.rand(1)[0])
    q7 = ql + (qu -ql)*(np.random.rand(1)[0])
    


    T_robot = T_base * Rz(q1) * Ry(q2) * Tz(d1) * Rz(q3) * Tx(d2) * Ry(q4) * Tx(d3) * Tz(d4) * Rz(q5) * Ry(q6) * Tx(d5) * Rz(q7)

    # print(Te)
    real_pos=np.hstack((T_robot.t,T_robot.rpy(order='zyx')))
    real_pos=np.array([real_pos])

    Corrected_model= T_base * Rz(q1) * Ry(q2-0.01) * Tz(d1_n) * Rz(q3) * Tx(d2_n) * Ry(q4) * Tx(d3_n) * Tz(d4_n) * Rz(q5) * Ry(q6) * Tx(d5_n) * Rz(q7)
    # print(Corrected_model)
    Corrected_pos=np.hstack((Corrected_model.t,Corrected_model.rpy(order='zyx')))
    Corrected_pos=np.array([Corrected_pos])
    
    Q=[q1,q2-0.01,q3,q4,q5,q6,q7]
    J=parameteric_jacobian(Q)

    delta_p=real_pos-Corrected_pos


    if i==0:
         Delta_ps=delta_p.T
         JJ=J
    else:
         Delta_ps=np.vstack((Delta_ps,delta_p.T))
         JJ=np.vstack((JJ,J))

d_pi=(np.linalg.inv(JJ.T@JJ))@((JJ.T)@(Delta_ps))
print(d_pi)
# print("pi_d2, delta_d2",d_pi[0][0],-0.05*d2)
# print("pi_d3, delta_d3",d_pi[2][0],-0.05*d3)
# print("pi_d4, delta_d4",d_pi[4][0],-0.05*d4)
# print("pi_d5, delta_d5",d_pi[5][0],-0.05*d5)

# pi_d2=D_pi[:,0]
# pi_d3=d_pi[11]
# pi_d4=d_pi[2]
# pi_d5=d_pi[3]

# pi_q2=d_pi[4]
# pi_q3=d_pi[5]
# pi_q4=d_pi[6]
# pi_q5=d_pi[7]
# corrected_model
#print(d_pi)




[[0.002429]
 [0.004154]
 [-0.0003236]
 [0.001341]
 [-0.008708]
 [ 0.00207]
 [0.005791]
 [ 0.01168]
 [-0.006125]
 [-0.0001113]]
